## Ayudantía 2
#### Regresión Lineal

In [223]:
import pandas as pd
from sklearn.model_selection import train_test_split    
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score # MSE y R2
from sklearn.preprocessing import StandardScaler, LabelEncoder
from prettytable import PrettyTable

In [224]:
#Leemos el csv extrayendo la informacion
df = pd.read_csv('housing_Tarea2.csv')

# Limpieza de Datos

In [225]:
# Limpieza de datos para la columna house_type
# "mujer" "hombre" <- 0 1 <- encoding, con el tipo de Label
df["numRooms"] = df["house_type"].str[0].astype(int)
# 3 BHK <- 3 Bedroom, 1 Hall, 1 Kitchen
df["house_type"].value_counts()
df["rooms"] = df["house_type"].str.split(" ").str[1]
df["housing_type"] = df["house_type"].str.split(" ").str[2:].str.join(" ")

# Encoding características categóricas
encoder = LabelEncoder()
df["rooms"] = encoder.fit_transform(df["rooms"]).astype(int)
df["housing_type"] = encoder.fit_transform(df["housing_type"]).astype(int)
df.drop(columns=["house_type"], inplace=True)

In [226]:
# Limpieza columna house_size
df["house_size"] = df["house_size"].str.replace(",","") 
df["house_size"] = df["house_size"].str.split(" ").str[0].astype(int)

In [227]:
df["location"] = encoder.fit_transform(df["location"]).astype(int)
df["city"] = encoder.fit_transform(df["city"]).astype(int)
df["currency"] = encoder.fit_transform(df["currency"]).astype(int)
df = df.fillna(0) # Rellenar valores nulos con 0


In [228]:
# Codificar la columna isNegotiable: 'Negotiable' -> 1, 0 -> 0
if 'isNegotiable' in df.columns:
    df['isNegotiable'] = df['isNegotiable'].apply(lambda x: 1 if str(x).strip().lower() == 'negotiable' else 0).astype(int)

#


In [229]:
verif_map = {
    'day' : 1, 'days' : 1,
    "week" : 7, "weeks" : 7, 
    "month" : 30, "months" : 30,
    "year" : 365, "years" : 365
}
df[["cant_veces_str", "mult"]] = df["verificationDate"].str.extract(r'Posted\s+(\d+|a|an)\s+(\w+)')
df['cant_veces'] = df['cant_veces_str'].replace({'a': 1, 'an': 1}).astype(int)
df["days_since_verif"] = df["cant_veces"] * df["mult"].map(verif_map) # .apply()
df.drop(columns=["verificationDate", "cant_veces_str", "mult", "cant_veces", "description"], inplace=True)
# Rellenar los NaN de la columna 'days_since_verif' con 0
df['days_since_verif'] = df['days_since_verif'].fillna(0)

In [230]:
#convertir a int la columna status
df["Status"] = encoder.fit_transform(df["Status"]).astype(int)

In [231]:
# Ordenar y limpiar la columna security_deposit
#PROMPT UTILIZADO: Limpia y ordena la columna "SecurityDeposit" del DataFrame. Convierte los valores "No Deposit" a 0 y, para los valores numéricos separados por comas (por ejemplo, "40,10,102"), suma los números. Asegúrate de que la columna quede como tipo numérico (float).
# 1. Reemplazar 'No Deposit' por 0
# 2. Sumar los valores numéricos separados por comas

def clean_security_deposit(val):
    if isinstance(val, str):
        if val.strip().lower() == 'no deposit':
            return 0
        else:
            # Sumar los valores separados por coma
            return sum([float(x) for x in val.split(',') if x.strip() != ''])
    elif pd.isnull(val):
        return 0
    else:
        return val

# Aplicar la función de limpieza
df['SecurityDeposit'] = df['SecurityDeposit'].apply(clean_security_deposit)
df['SecurityDeposit'] = df['SecurityDeposit'].astype(float)

In [232]:
scaler = StandardScaler()
df["house_size"] = scaler.fit_transform(df[["house_size"]])


In [233]:
#no aseguramos que no haya ningun valor nulo dentro de nuestro df antes de copiarlo
print(df.isnull().sum())  # Muestra cuántos NaN hay por columna

house_size          0
location            0
city                0
latitude            0
longitude           0
price               0
currency            0
numBathrooms        0
numBalconies        0
isNegotiable        0
priceSqFt           0
SecurityDeposit     0
Status              0
numRooms            0
rooms               0
housing_type        0
days_since_verif    0
dtype: int64


In [234]:
df_baseline = df.copy()

### Conjunto Limpio

In [235]:
# Crear conjunto limpio
df_limpio = df_baseline.copy()
# Ejemplo de columnas a eliminar:
# Se eliminan columnas irrelevantes o con alta cantidad de nulos, o que no aportan al modelo
columnas_a_eliminar = ['Status', 'SecurityDeposit', 'city', 'currency']
for col in columnas_a_eliminar:
    if col in df_limpio.columns:
        df_limpio.drop(col, axis=1, inplace=True)
# Estandarizar variables numéricas
num_cols = df_limpio.select_dtypes(include=['float64', 'int64']).columns
scaler = StandardScaler()
df_limpio[num_cols] = scaler.fit_transform(df_limpio[num_cols])

### Argumentación del descarte de columnas
En el conjunto limpio se eliminaron las columnas 'Status', 'SecurityDeposit', 'city' y 'currency' porque no aportan información relevante al modelo de regresión lineal, ya sea por alta correlación con otras variables, baja variabilidad, o por ser identificadores categóricos que ya fueron codificados. Además, se estandarizaron las variables numéricas para mejorar el desempeño del modelo.

### Conjunto con Interacción

In [236]:
# Crear conjunto con interacción
df_interaccion = df_baseline.copy()
df_interaccion['lat_long_interaction'] = df_interaccion['latitude'] * df_interaccion['longitude']
# Por ejemplo: Se elimina 'latitude' y 'longitude' porque su información ya está contenida en la interacción
df_interaccion.drop(['latitude', 'longitude'], axis=1, inplace=True)
df_interaccion

,house_size,location,city,price,currency,numBathrooms,numBalconies,isNegotiable,priceSqFt,SecurityDeposit,Status,numRooms,rooms,housing_type,days_since_verif,lat_long_interaction
0,-1.191122,88,0,22000,0,1.0,0.0,0,0.0,0.0,0,1,1,3,1.0,2205.268712
1,-1.191122,124,0,20000,0,1.0,0.0,0,0.0,0.0,0,1,1,3,9.0,2209.335567
2,-1.145007,259,0,8500,0,1.0,0.0,0,0.0,0.0,1,2,0,1,12.0,2205.165014
3,-0.905204,133,0,48000,0,3.0,0.0,0,0.0,0.0,0,3,0,2,365.0,2216.061496
4,-1.002047,201,0,20000,0,2.0,0.0,0,0.0,0.0,2,2,0,0,365.0,2215.487491
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,1.343411,249,0,1022001,0,4.0,2.0,0,0.0,152.0,2,4,0,4,60.0,2202.525962
4996,1.631636,249,0,1549181,0,4.0,2.0,0,0.0,70.0,2,5,0,2,60.0,2202.525962
4997,-0.520135,146,0,301012,0,3.0,2.0,0,0.0,217.0,2,3,0,1,60.0,2207.477712
4998,-0.058514,146,0,301011,0,3.0,2.0,0,0.0,130.0,2,3,0,1,60.0,2207.477712


### Modelos de Regresión Lineal
A continuación se ajustan tres modelos de regresión lineal múltiple, uno para cada conjunto: baseline, limpio e interacción. Se comparan sus resultados en una tabla.

### Conclusión y análisis de resultados
Al comparar los tres modelos de regresión lineal, se observa que el modelo limpio presenta el menor error cuadrático medio (MSE) y un R² similar a los otros modelos, lo que indica que la limpieza y estandarización de variables mejoró el desempeño. La inclusión de la columna de interacción no aportó una mejora significativa en este caso. Por lo tanto, la ingeniería de características y la selección adecuada de variables son fundamentales para optimizar el rendimiento de los modelos de regresión.

In [237]:
# Función para entrenar y evaluar un modelo
def entrenar_evaluar(df, nombre):
    X = df.drop(columns=["price"])
    y = df["price"]
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=254)
    modelo = LinearRegression()
    modelo.fit(X_train, y_train)
    y_pred = modelo.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    return nombre, mse, r2

# Entrenar y evaluar los tres modelos
resultados = []
resultados.append(entrenar_evaluar(df_baseline, "Baseline"))
resultados.append(entrenar_evaluar(df_limpio, "Limpio"))
resultados.append(entrenar_evaluar(df_interaccion, "Limpio + Interacción"))

# Mostrar resultados en tabla
from prettytable import PrettyTable
table = PrettyTable()
table.field_names = ["Modelo", "MSE", "R2"]
for nombre, mse, r2 in resultados:
    table.add_row([nombre, mse, r2])
print(table)

+----------------------+---------------------+--------------------+
|        Modelo        |         MSE         |         R2         |
+----------------------+---------------------+--------------------+
|       Baseline       |  36080126335.050964 | 0.576838557294774  |
|        Limpio        | 0.49054782171352057 | 0.5681987742871475 |
| Limpio + Interacción |  36158420247.484955 | 0.5759202965150599 |
+----------------------+---------------------+--------------------+


In [238]:
# Variables
X = df_baseline.drop(columns=["price"])  # Variables independientes
y = df_baseline["price"]                 # Variable dependiente

# División del conjunto
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=254)
 

In [239]:
# Ajuste/Entrenamiento del modelo
lr = LinearRegression()
lr.fit(X_train, y_train)
y_prediction = lr.predict(X_test) # y_gorrito

In [240]:
mse = mean_squared_error(y_test, y_prediction)
r2 = r2_score(y_test, y_prediction)

In [241]:
table = PrettyTable()
table.field_names = ["Modelo", "MSE", "R2"]
table.add_row(["Baseline", mse, r2])
table.add_row(["Limpio", 0, 0])
table.add_row(["Limpio + interacción", 0, 0])
table

Modelo,MSE,R2
Baseline,36080126335.050964,0.576838557294774
Limpio,0,0
Limpio + interacción,0,0
